# **Final: Experiments**

## **1. Read the datasets**

In [4]:
import pandas as pd

dflist_areas = []
areas = ['ingenieria', 'negocios','estudioscreativos', 'salud', 'cienciassociales', 'ambienteconstruido']

for elem in areas:
    df_area = pd.read_csv("C:/Users/Milara/..._"+elem+"_v1.csv",encoding="ISO-8859-1")
    dflist_areas.append(df_area)

## **2. Remove variables not used (NaN removed)**

In [5]:
#df_selected_features = pd.read_csv("C:/Users/Milara/...",encoding="UTF-8")

#for i in range(len(areas)):
#    all_features = df_selected_features[areas[i]].dropna().to_list()
#    df_wo_na_all = dflist_areas[i][all_features].dropna()
#    dflist_areas[i] = df_wo_na_all

## **3. Create binary output (balanced classes)**

In [6]:
def update(val,threshold):
    if val < threshold:
        return 0.0
    elif val >= threshold:
        return 1.0
    else:
        return np.nan

for elem in dflist_areas:
    median = elem['For_Prom'].median()
    level = elem['For_Prom'].apply(lambda x: update(x,median))
    #Inserting the values in the df
    elem.insert(elem.shape[1],'Binary_out',level.values)
   

# **4. Select variables based on dataset**

In [8]:

def remove_var(prefix,lista):
    res = lista.copy()
    for elem in prefix:
        for i in range(len(lista)):
            value = lista[i].find(elem)
            if value >= 0:
                res.remove(lista[i])
    return res

def keep_var(prefix,lista):
    res = []
    res.append(lista[-3]) # to add calpromedioprepa
    for elem in prefix:
        for i in range(len(lista)):
            value = lista[i].find(elem)
            if value >= 0:
                res.append(lista[i])
    res.append(lista[-2])
    res.append(lista[-1])
    return res

def select_database(i, df): #requiere los nombres de las columnas en lista ..list_areas[0].columns.to_list()
    features = df.columns.to_list()
    if i == 0:
        #Caso 0 ->  PAA
        list_var = keep_var(['Puntaje'], features)
    elif i == 1:
        # Caso 1 ->  Eva
        list_var = keep_var(['Eva'], features)
    elif i == 2 :
        # Caso 2 ->  Senti
        list_var = keep_var(['_n'], features)
    elif i == 3:
        # Caso 3 -> Ord 
        list_var = keep_var(['_ord'], features)
    elif i == 4:
        # Caso 4 -> eval + paa
        list_var = keep_var(['Eva','Puntaje'], features)
    elif i == 5:
        # Caso 5 -> eval + senti
        list_var = keep_var(['Eva','_n'], features)
    elif i == 6:
        # Caso 6 -> eval + ord
        list_var = keep_var(['Eva','_ord'], features)
    elif i == 7:
        # Caso 7 -> paa + senti
        list_var = keep_var(['Puntaje','_n'], features)
    elif i == 8:
        # Caso 8 -> paa + ord
        list_var = keep_var(['Puntaje','_ord'], features)
    elif i == 9:
        # Caso 9 -> senti + ord
        list_var = keep_var(['_n','_ord'], features)
    elif i == 10:
        # Caso 10 -> eval + paa + senti
        list_var = keep_var(['Eva','Puntaje','_n'], features)
    elif i == 11:
        # Caso 11 -> eval + paa + ord
        list_var = keep_var(['Eva','Puntaje','_ord'], features)
    elif i == 12:
        # Caso 12 -> eval + senti + ord
        list_var = keep_var(['Eva','_n','_ord'], features)
    elif i == 13:
        # Caso 13 -> paa + senti + ord
        list_var = keep_var(['Puntaje','_n','_ord'], features)
    elif i == 14:
        # Caso 14 -> eval + paa + senti + ord
        list_var = features
    elif i == 15:
        # Caso 14 -> eval without CalPrepa
        list_var = keep_var(['Eva'], features)
        list_var.remove('CalPromedioPrepa')
    
    list_var.remove('For_Prom')
    
    return df[list_var]

# **5 .Feature Selection**

## **PCA**

In [9]:
#PCA
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np

def PCA_selection(df, corrThres): #requiere df sin NaN
    all_features = df.columns.to_list()
    scaled_data = preprocessing.scale(df.loc[:,all_features[:-1]])
    pca = PCA()
    pca.fit(scaled_data)
    pca_data = pca.transform(scaled_data)
    
    per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
    labels = [ 'PC' + str(x) for x in range(1, len(per_var) +1 )]
    pca_df_1 = pd.DataFrame(pca_data, index = df.loc[:,all_features[:-1]].index, columns = labels)
    df_feat_pcs = pd.concat([df[all_features[:-1]],pca_df_1[['PC1']]], axis = 1)
    corr = df_feat_pcs.corr()

    #IMPORTANT CELL WHERE THE VARIABLES ARE SELECTED BASED ON THE ABS OF 0.5 FOR CORRELATION
    corr_reduced = corr.loc[all_features[:-1]]
    features = corr_reduced[(corr_reduced['PC1'] < -corrThres)  | (corr_reduced['PC1'] > corrThres)].sort_values(by ='PC1').index.to_list()
    features.append(all_features[-1])
    df_selected = df[features]
    
    return df_selected

### *Testing it*

In [8]:
ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(15, ing) #Select the database
ing_red.info()
#ing_sel = PCA_selection(ing_red, 0.37) #Apply PCA Selection
#ing_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 6 columns):
Eva_ini_UMM1    1862 non-null float64
Eva_ini_UMM2    1862 non-null float64
Eva_ini_UMF1    1862 non-null float64
Eva_ini_UMF2    1862 non-null float64
Eva_ini_UMC1    1862 non-null float64
Binary_out      1862 non-null float64
dtypes: float64(6)
memory usage: 87.4 KB


## **T-test (only the variables where there is significant difference between the two groups)**

In [10]:
from scipy import stats
import numpy as np

def Ttest_selection(df,levelSignificance):
    df_red_1 = df[df['Binary_out'] == 1.0]
    df_red_0 = df[df['Binary_out'] == 0.0]

    features = df.columns.to_list()[:-1]
    stat = []
    pval = []

    for elem in features:
        stat.append(stats.ttest_ind(df_red_1[elem], df_red_0[elem], equal_var = False).statistic)
        pval.append(round(stats.ttest_ind(df_red_1[elem], df_red_0[elem], equal_var = False).pvalue,5))

    d = {'Feature': features, 'Statistic':stat , 'P-value':pval}
    df_ttest = pd.DataFrame(data=d)
    #print(df_ttest)
    ordered = df_ttest.sort_values(by = 'Statistic', ascending = False)
    significance = ordered[ordered['P-value'] < levelSignificance]
    #print(significance)
    sel_vars = significance['Feature'].to_list()
    sel_vars.append('Binary_out')
    
    return(df[sel_vars])

### *Testing it*

In [9]:
ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(5, ing) #Select the database
Ttest_selection(ing_red,0.05).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 9 columns):
CalPromedioPrepa    1862 non-null float64
Eva_ini_UMM1        1862 non-null float64
Eva_ini_UMF1        1862 non-null float64
Eva_ini_UMC1        1862 non-null float64
Eva_ini_UMM2        1862 non-null float64
Eva_ini_UMF2        1862 non-null float64
confianza_n         1862 non-null int64
miedo_n             1862 non-null int64
Binary_out          1862 non-null float64
dtypes: float64(7), int64(2)
memory usage: 131.0 KB


## **RFE**

In [11]:
#scaling and centering the data
from sklearn import preprocessing
# Feature ranking with recursive feature elimination and cross-validation
from sklearn.feature_selection import RFECV

def RFE_selection(df,model):
    
    if len(df.columns) > 2:
        predictors = df.columns.to_list()[:-1]
        df_sc = preprocessing.scale(df.loc[:,predictors])

        X = pd.DataFrame(df_sc, columns = predictors)
        y = df['Binary_out']

        rfecv = RFECV(estimator=model, step=1, cv=10, scoring='accuracy')
        rfecv.fit(X, y)

        #print("Optimal number of features: %d" % rfecv.n_features_)
        #print('Selected features: %s' % list(X.columns[rfecv.support_]))
        sel_vars = list(X.columns[rfecv.support_])
        sel_vars.append('Binary_out')
        return df[sel_vars]
    else:
        return df

### *Testing it*

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier


ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(0, ing) #Select the database
RFE_selection(ing_red,LogisticRegression()).info() 
#RFE_selection(ing_red,tree.DecisionTreeClassifier()).info() 
#RFE_selection(ing_red,RandomForestClassifier()).info() 
#RFE_selection(ing_red,svm.SVC(kernel="linear")).info()

#No funcionan****
#RFE_selection(ing_red,KNeighborsClassifier()).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CalPromedioPrepa           1862 non-null   float64
 1   Puntaje Seccion 14 Examen  1862 non-null   float64
 2   Puntaje Seccion 19 Examen  1862 non-null   float64
 3   Binary_out                 1862 non-null   float64
dtypes: float64(4)
memory usage: 58.3 KB


## **6. Select Feature Selection**

In [13]:
def select_variables(i,df,corrThres,levelSignificance):
    if i == 0:
        df_res = PCA_selection(df,corrThres)
    elif i == 1:
        df_res = Ttest_selection(df,levelSignificance)
    elif i == 2:
        df_res = df 
    return df_res

def use_rfe(df,model,hymodel, area, database):
    df_rfe = RFE_selection(df,model)
    predictors = df_rfe.columns.to_list()[:-1]
    X = preprocessing.scale(df_rfe.loc[:,predictors])
    y = df_rfe['Binary_out']
    result = hymodel(X,y,area, database, "RFE", predictors)
    return result

### *Testing it*

In [49]:
ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(1, ing) #Select the database
select_variables(2,ing_red,0.37,10).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 7 columns):
CalPromedioPrepa    1862 non-null float64
Eva_ini_UMM1        1862 non-null float64
Eva_ini_UMM2        1862 non-null float64
Eva_ini_UMF1        1862 non-null float64
Eva_ini_UMF2        1862 non-null float64
Eva_ini_UMC1        1862 non-null float64
Binary_out          1862 non-null float64
dtypes: float64(7)
memory usage: 101.9 KB


In [54]:
ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(0, ing) #Select the database

use_rfe(ing_red,LogisticRegression(),logit_model, 'Ingenieria', 'PAA')
#use_rfe(ing_red,tree.DecisionTreeClassifier(),dt_model, 'Ingenieria', 'PAA')
#use_rfe(ing_red,RandomForestClassifier(),rf_model, 'Ingenieria', 'PAA')
#use_rfe(ing_red,svm.SVC(kernel="linear"),svm_model, 'Ingenieria', 'PAA')

[[0.69745138 0.48394961 0.23919562]] ['CalPromedioPrepa', 'Puntaje Seccion 14 Examen', 'Puntaje Seccion 19 Examen']


,Area,Database,Feature Selection,Model,Accuracy,Variables
0,Ingenieria,PAA,RFE,LogReg,0.6974,"[CalPromedioPrepa, Puntaje Seccion 14 Examen, ..."


## **7. Models**

In [15]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.naive_bayes import GaussianNB

def logit_model(X,y, area, db, fe, predictors):
    clf = LogisticRegression(random_state=0)
    cv_method = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=5)
    cv_results_full = cross_val_score(estimator=clf, X=X, y=y, cv=cv_method, scoring='accuracy')
    accuracy = cv_results_full.mean().round(4)
    
    #clf_print = LogisticRegression(random_state=0).fit(X, y)#to use only when getting coefficients
    #print(clf_print.coef_,predictors)
    
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] ,
                                "Model":["LogReg"], "Accuracy":[accuracy], "Variables": [predictors]})

def knn_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_knn = {'n_neighbors': [10,50,100], 'p': [1, 2]}
    grs_knn = GridSearchCV(estimator=KNeighborsClassifier(),
                      param_grid=param_knn,
                      cv=cv_method,
                      verbose=False,
                      scoring='accuracy')
    grs_knn.fit(X, y)
    accuracy = grs_knn.best_score_.round(4)
    print(grs_knn.best_params_," KNN ", accuracy)
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["KNN"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

def svm_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = {'kernel': ['rbf', 'poly', 'linear']} 
    grs_svc = GridSearchCV(estimator=SVC(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring='accuracy')
    grs_svc.fit(X, y)
    accuracy = grs_svc.best_score_.round(4)
    print(grs_svc.best_params_," SVM ", accuracy)
    #svmf_clf = svm.SVC(kernel ='linear')
    #svmf_clf.fit(X,y)
    #print(svmf_clf.coef_,predictors) #to comment when not getting the coefficents of the models
    
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["SVM"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

def dt_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = {'criterion': [ 'gini', 'entropy'], 
              'max_depth': [2, 5, 10], 
              'random_state': [17]} 
    grs_tree = GridSearchCV(estimator=tree.DecisionTreeClassifier(),
                      param_grid=param_grid,
                      cv=cv_method,
                      verbose=False,
                      scoring='accuracy')
    grs_tree.fit(X, y)
    accuracy = grs_tree.best_score_.round(4)
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["DT"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

def rf_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = {'criterion': [ 'gini', 'entropy'], 
                  'max_depth': [2,5], #2 y 5 
                  'random_state': [17],
                 'n_estimators': [50,100]} 
    grs_rf = GridSearchCV(estimator=RandomForestClassifier(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring='accuracy')
    grs_rf.fit(X, y)
    accuracy = grs_rf.best_score_.round(4)
    print(grs_rf.best_params_," RF ", accuracy)
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["RF"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

def xgb_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = {'learning_rate': [0.05,0.01], #0.05 y 0.01
                  'max_depth': [8,32], #8 y 32
                  'random_state': [17],
                  'min_child_weight':[1,6],
                 'n_estimators': [100],
                 'subsample': [0.5],
                 } 

    grs_xgb = GridSearchCV(estimator= xgb.XGBClassifier(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring='accuracy')
    grs_xgb.fit(X, y)
    accuracy = grs_xgb.best_score_.round(4)
    print(grs_xgb.best_params_, "XGB", accuracy)
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["XGB"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

def nb_model(X,y, area, db, fe, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = {'priors': [None] } 
    grs_nb = GridSearchCV(estimator= GaussianNB(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring='accuracy')
    grs_nb.fit(X, y)
    accuracy = grs_nb.best_score_.round(4)
    print(grs_nb.best_params_, " NB ", accuracy)
    return pd.DataFrame(data = {"Area": [area], "Database":[db], "Feature Selection":[fe] , "Model":["NB"],
                                "Accuracy":[accuracy], "Variables": [predictors]})

### *Testing it*

In [16]:
ing = dflist_areas[0].copy() #Select the area
ing_red = select_database(14, ing) #Select the database
df_test = select_variables(1,ing_red,0.37,0.05)
#df_test.info()

predictors = df_test.columns.to_list()[:-1]
df_sc_t = preprocessing.scale(df_test.loc[:,predictors])

X = df_sc_t.copy()
y = df_test['Binary_out']

#logit_model(X,y, "Engineering", "PAA", "Ttest",predictors)
#knn_model(X,y, "Engineering", "PAA", "Ttest",predictors)
#svm_model(X,y, "Engineering", "PAA", "Ttest",predictors)
#dt_model(X,y, "Engineering", "PAA", "Ttest",predictors)
rf_model(X,y, "Engineering", "PAA", "Ttest",predictors)
#xgb_model(X,y, "Engineering", "PAA", "Ttest",predictors)
#nb_model(X,y, "Engineering", "PAA", "Ttest",predictors)

,Area,Database,Feature Selection,Model,Accuracy,Variables
0,Engineering,PAA,Ttest,RF,0.7266,"[CalPromedioPrepa, Eva_ini_UMM1, Puntaje Secci..."


## **8. Experiments with hypertunning parameters**

In [32]:
from sklearn import preprocessing

df_results = pd.DataFrame(data = {"Area": [], "Database":[], "Feature Selection":[] , "Model":[], "Accuracy":[], "Variables":[]})
databases = ["Paa","Eval","Senti","Ord","Eval + Paa","Eval + Senti","Eval + Ord","Paa + Senti","Paa + Ord",
             "Senti + Ord","Eval + Paa + Senti","Eval + Paa + Ord","Eval + Senti + Ord","Paa + Senti + Ord","Eval + Paa + Senti + Ord",'Eval No CalPrepa']
fselection = ["PCA","Ttest","None"]

for i in range(5,6,1): #6 Iteration over all the areas
    for j in range(10,13,1): #16 Iteration over all the databases
        df_db = select_database(j, dflist_areas[i])
        #df_db.insert(3,column = 'Noise', value = np.random.randint(1,1000,df_db.shape[0]))

        print(df_db.info())
        
        for k in range(1,2,1):# 2 Iteration over the feature selection models, avoid PCA only T-test and None
            df_fe = select_variables(k,df_db,0.37,0.05) 
            
            if len(df_fe.columns) > 1 :
                predictors = df_fe.columns.to_list()[:-1]
                df_sc = preprocessing.scale(df_fe.loc[:,predictors])
                X = df_sc.copy()
                y = df_fe['Binary_out']
                
                #a = logit_model(X,y,areas[i], databases[j], fselection[k],predictors)
                b = knn_model(X,y,  areas[i], databases[j], fselection[k],predictors)
                #c = svm_model(X,y,  areas[i], databases[j], fselection[k],predictors)
                #d = dt_model(X,y,   areas[i], databases[j], fselection[k],predictors)
                #e = rf_model(X,y,   areas[i], databases[j], fselection[k],predictors)
                #f = xgb_model(X,y,  areas[i], databases[j], fselection[k],predictors)
                g = nb_model(X,y,   areas[i], databases[j], fselection[k],predictors)
                
            else:# In the case Ttest does not find any variable
                a = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["LogReg"], "Accuracy":[0], "Variables":['NONE']})
                b = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["KNN"], "Accuracy":[0], "Variables":['NONE']})
                c = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["SVM"], "Accuracy":[0], "Variables":['NONE']})
                d = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["DT"], "Accuracy":[0], "Variables":['NONE']})
                e = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["RF"], "Accuracy":[0], "Variables":['NONE']})
                f = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["XGB"], "Accuracy":[0], "Variables":['NONE']})
                g = pd.DataFrame(data = {"Area": [areas[i]], "Database":[databases[j]], "Feature Selection":[fselection[k]] , "Model":["NB"], "Accuracy":[0], "Variables":['NONE']})
            
            #df_results = pd.concat([df_results,a,b,c,d,e], axis = 0)
            df_results = pd.concat([df_results,b,g], axis = 0)
        
        #RFE
        #a = use_rfe(df_db,LogisticRegression(),logit_model, areas[i], databases[j])
        #b = use_rfe(df_db,svm.SVC(kernel="linear"),svm_model, areas[i], databases[j])
        #c = use_rfe(df_db,tree.DecisionTreeClassifier(),dt_model, areas[i], databases[j])
        #e = use_rfe(df_db,RandomForestClassifier(),rf_model, areas[i], databases[j])
        #f = use_rfe(df_db,xgb.XGBClassifier(),xgb_model, areas[i], databases[j])
        
        #df_results = pd.concat([df_results,a,b,c,e], axis = 0)
        #df_results = pd.concat([df_results,b], axis = 0)
        
        print(areas[i] , databases[j], df_db.shape)

#df_results.head()            

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 16 columns):
CalPromedioPrepa             120 non-null float64
Eva_ini_UMM2                 120 non-null float64
Eva_ini_UMM3                 120 non-null float64
Eva_ini_UMM4                 120 non-null float64
Eva_ini_UMF1                 120 non-null float64
Eva_ini_UMF2                 120 non-null float64
Eva_ini_UMC1                 120 non-null float64
Eva_ini_UMC2                 120 non-null float64
Puntaje Seccion 8 Examen     120 non-null float64
Puntaje Seccion 13 Examen    120 non-null float64
Puntaje Seccion 14 Examen    120 non-null float64
Puntaje Seccion 19 Examen    120 non-null float64
miedo_n                      120 non-null int64
sorpresa_n                   120 non-null int64
enojo_n                      120 non-null float64
Binary_out                   120 non-null float64
dtypes: float64(14), int64(2)
memory usage: 15.1 KB
None
{'n_neighbors': 100, 'p': 1}  KNN  0.7417


In [47]:
df_results.to_csv("C:/Users/Milara/...", encoding="ISO-8859-1", index = False) #thus it is necessary to save it as the usual type of coding

## **9. Noise Testing**

In [12]:
#Read the file with the data of the best experiments
df_exp = pd.read_csv("C:/Users/Milara/...",encoding="ISO-8859-1")
df_exp.head()

,Area,Database,Feature Selection,Model,Parameters,Accuracy,Variables
0,ingenieria,Eval + Paa,None,RF,"{'criterion': 'gini', 'max_depth': 5, 'n_estim...",0.73,"['CalPromedioPrepa', 'Eva_ini_UMM1', 'Eva_ini_..."
1,ingenieria,Eval + Paa,RFE,LogReg,NOT,0.73,"['CalPromedioPrepa', 'Eva_ini_UMM1', 'Eva_ini_..."
2,negocios,Paa + Senti + Ord,RFE,LogReg,NOT,0.70,"['CalPromedioPrepa', 'Puntaje Seccion 8 Examen..."
3,negocios,Eval + Paa + Senti + Ord,RFE,SVM,{'kernel': 'linear'},0.70,"['P6_ord', 'LABORAL_ACTU_ord', 'confianza_n', ..."
4,estudioscreativos,Senti,Ttest,NB,{'priors': None},0.63,"['CalPromedioPrepa', 'confianza_n']"


### Function to help reading the experiments

In [22]:
#Transform the string of variables into a list
def string_2list(string):
    a = string
    char_remove = ['[',']','\'']
    for elem in char_remove:
        a = a.replace(elem, '')
    return a.split(sep = ', ')

### Model creation

In [47]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.naive_bayes import GaussianNB
import ast
import shap

scoring_metric = 'accuracy' # used to get the precision and recall from the models

def logreg_model_S(X,y, area, parameters, predictors, coefficients = False):
    clf = LogisticRegression(random_state=0)
    cv_method = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=5)
    cv_results_full = cross_val_score(estimator=clf, X=X, y=y, cv=cv_method, scoring= scoring_metric)
    accuracy = cv_results_full.mean().round(4)
    
    coefficients_clf = []
    if coefficients:
        clf_print = LogisticRegression(random_state=0).fit(X, y)#to use only when getting coefficients
        coefficients_clf = clf_print.coef_

    return pd.DataFrame(data = {"Area": [area],"Model":["LogReg"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [cv_results_full],
                       "Variables" : [predictors], "Coefficients" : [coefficients_clf]})

def knn_model_S(X,y, area, parameters, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters
    grs_knn = GridSearchCV(estimator=KNeighborsClassifier(),
                      param_grid=param_grid,
                      cv=cv_method,
                      verbose=False,
                      scoring= scoring_metric)
    grs_knn.fit(X, y)
    accuracy = grs_knn.best_score_.round(4)
    #print(grs_knn.best_params_," KNN ", accuracy)
    
    knn_cv = pd.DataFrame(grs_knn.cv_results_)
    filteredList = list(filter(lambda x: x[0:5] == 'split', knn_cv.columns))
    
    return pd.DataFrame(data = {"Area": [area],"Model":["KNN"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [knn_cv[filteredList].stack().values],
                       "Variables" : [predictors], "Coefficients" : ['NOT']})


def svm_model_S(X,y, area, parameters, predictors, coefficients = False):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters
    grs_svc = GridSearchCV(estimator=SVC(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring=scoring_metric)
    grs_svc.fit(X, y)
    accuracy = grs_svc.best_score_.round(4)
    #print(grs_svc.best_params_," SVM ", accuracy)
    
    coefficients_svc = []
    if coefficients:
        svc_print = svm.SVC(kernel ='linear').fit(X, y)#to use only when getting coefficients
        coefficients_svc = svc_print.coef_
    
    svc_cv = pd.DataFrame(grs_svc.cv_results_)
    filteredList = list(filter(lambda x: x[0:5] == 'split', svc_cv.columns))
    
    return pd.DataFrame(data = {"Area": [area],"Model":["SVM"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [svc_cv[filteredList].stack().values],
                       "Variables" : [predictors], "Coefficients" : [coefficients_svc]})

def dt_model_S(X,y, area, parameters, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters
    grs_tree = GridSearchCV(estimator=tree.DecisionTreeClassifier(),
                      param_grid=param_grid,
                      cv=cv_method,
                      verbose=False,
                      scoring=scoring_metric)
    grs_tree.fit(X, y)
    accuracy = grs_tree.best_score_.round(4)
    
    return pd.DataFrame(data = {"Area": [area],"Model":["DT"], "Parameters": [parameters], "Accuracy":[accuracy],
                                "Variables": [predictors],"Coefficients" : ['NOT']})

def rf_model_S(X,y, area, parameters, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters 
    grs_rf = GridSearchCV(estimator=RandomForestClassifier(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring=scoring_metric)
    grs_rf.fit(X, y)
    accuracy = grs_rf.best_score_.round(4)
    #print(grs_rf.best_params_," RF ", accuracy)
    
    rf_cv = pd.DataFrame(grs_rf.cv_results_)
    filteredList = list(filter(lambda x: x[0:5] == 'split', rf_cv.columns))
    
    return pd.DataFrame(data = {"Area": [area],"Model":["RF"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [rf_cv[filteredList].stack().values],
                       "Variables" : [predictors],"Coefficients" : ['NOT']})

def xgb_model_S(X,y, area, parameters, predictors, coefficients = False):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters 

    grs_xgb = GridSearchCV(estimator= xgb.XGBClassifier(use_label_encoder = False),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring=scoring_metric)
    grs_xgb.fit(X, y)
    accuracy = grs_xgb.best_score_.round(4)
    #print(grs_xgb.best_params_, "XGB", accuracy)
    
    
    coefficients_xgb = []
    #if coefficients:
     #   xgb_print = xgb.XGBClassifier(b).fit(X, y)#to use only when getting coefficients NOT WORKING WITH PARAMETERS AS DICT
        #coefficients_xgb = xgb_print.feature_importances_
    
    xgb_cv = pd.DataFrame(grs_xgb.cv_results_)
    filteredList = list(filter(lambda x: x[0:5] == 'split', xgb_cv.columns))
    
    return pd.DataFrame(data = {"Area": [area],"Model":["XGB"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [xgb_cv[filteredList].stack().values],
                       "Variables" : [predictors], "Coefficients" : [coefficients_xgb]})

def nb_model_S(X,y, area, parameters, predictors):
    cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=5)
    param_grid = parameters
    grs_nb = GridSearchCV(estimator= GaussianNB(),
                          param_grid=param_grid,
                          cv=cv_method,
                          verbose=False,
                          scoring=scoring_metric)
    grs_nb.fit(X, y)
    accuracy = grs_nb.best_score_.round(4)
    #print(grs_nb.best_params_, " NB ", accuracy)

    nb_cv = pd.DataFrame(grs_nb.cv_results_)
    filteredList = list(filter(lambda x: x[0:5] == 'split', nb_cv.columns))
    
    return pd.DataFrame(data = {"Area": [area],"Model":["NB"], "Parameters": [parameters], "Accuracy":[accuracy], "CV Results": [nb_cv[filteredList].stack().values],
                       "Variables" : [predictors],"Coefficients" : ['NOT']})


### Functions to run the comparision between experiments with and without noise

In [15]:
from scipy import stats
import random

def run_cv(df_inuse,id_area,noise_flag = False):
    """
    Function to run the experiments based on their parameters and noise and return the cv results
    Input: 
        df with the experiments to be run,
        value of id of the area
        boolean value for the flag
    Output:
        df with the cross validation scores
    """
    df_inuse = df_inuse[df_inuse['Database'] != 'Eval No CalPrepa'].drop(['Feature Selection', 'Accuracy','Database'], axis = 1) #remove experiment not needed
    df_db = dflist_areas[id_area]# Get the full database

    #Store the data to configure the experiment
    models = df_inuse['Model'].tolist()
    variables = df_inuse['Variables'].tolist()
    parameters = df_inuse['Parameters'].tolist()
    
    list_results = list()

    #Compute the models
    for i in range(len(models)):
        list_variables = string_2list(variables[i]) # get the list of variables to be used in the model

        #get the parameters for the model
        if parameters[i] != 'NOT':
            dict_param = ast.literal_eval(parameters[i])

            for elem in dict_param.keys(): #transform to list the values of each key
                dict_param[elem] = [dict_param[elem]]
        else:
            dict_param = 'NA'

        #Clean the db with the selected variables
        data_samples = df_db.loc[:,list_variables]
        
        if noise_flag:#used only if flag enabled
            #np.random.seed(100)
            data_samples.insert(0,column = 'Noise', value = np.random.randint(1,1000,df_db.shape[0])) # Noise added
            
        X = preprocessing.scale(data_samples)
        y = df_db['Binary_out']
        model = models[i].lower() 

        #Compute the model
        if model == 'rf':
            rf_res = rf_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(rf_res)
        elif model == 'logreg':
            log_res = logreg_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(log_res)
        elif model == 'svm':
            svm_res = svm_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(svm_res)
        elif model == 'knn':
            knn_res = knn_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(knn_res)
        elif model == 'nb':
            nb_res = nb_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(nb_res)
        elif model == 'xgb':
            xgb_res = xgb_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(xgb_res)

    return pd.concat(list_results, axis = 0)


def paired_ttest(base_cv,noise_cv):
    """
    Function that is used to compare the mean using paired t test
    Input: two df 
    Output: model with less difference in the means
    """
    model_winner = ""
    pvalue_winner = 0
    variables_winner = list()
    accuracy_winner = 0
    parameters_winner = dict()

    for i in range(len(base_cv.index)):
        statistic, pvalue = stats.ttest_rel(base_cv['CV Results'][i], noise_cv['CV Results'][i])
        if pvalue_winner < pvalue:
            pvalue_winner  = pvalue
            model_winner = base_cv['Model'][i]
            variables_winner = base_cv['Variables'][i]
            accuracy_winner = base_cv['Accuracy'][i]
            parameters_winner = base_cv['Parameters'][i]

    return(base_cv['Area'][i],variables_winner,model_winner,pvalue_winner,accuracy_winner,parameters_winner)

### *Testing it*

In [17]:
areas = ['ingenieria', 'negocios','estudioscreativos', 'salud', 'cienciassociales', 'ambienteconstruido']
id_area = 0
winner_list = []
repeat_list = []

for i in range(3):
    for i in range(len(areas)):
        #Get the data for the experiment
        df_inuse = df_exp[df_exp['Area'] == areas[i]] # Select the area
        #compute the cross validation results
        base_cv = run_cv(df_inuse,i).reset_index().drop('index', axis = 1)
        noise_cv = run_cv(df_inuse,i,True).reset_index().drop('index', axis = 1)
        #compare the cross validation results with paired t test
        warea, wvariables, wmodel, wpvalue, waccuracy, wparameters = paired_ttest(base_cv,noise_cv)
        winner = pd.DataFrame(data = {"Area": [warea],"Model":[wmodel], "Parameters": [wparameters], "Accuracy":[waccuracy], "Variables" : [wvariables]})
        winner_list.append(winner)

    winner_df = pd.concat(winner_list, axis = 0)
    repeat_list.append(winner_df)
    
#winner_df.to_csv("C:/Users/Milara/..., encoding="ISO-8859-1", index = False) #thus it is necessary to save it as the usual type of coding

### Results

In [18]:
repeat_list[0].head(6)

,Area,Model,Parameters,Accuracy,Variables
0,ingenieria,LogReg,NA,0.7256,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,negocios,LogReg,NA,0.7034,"[CalPromedioPrepa, Puntaje Seccion 8 Examen, P..."
0,estudioscreativos,SVM,{'kernel': ['linear']},0.6295,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,salud,XGB,"{'learning_rate': [0.05], 'max_depth': [8], 'm...",0.6648,"[CalPromedioPrepa, enojo_n]"
0,cienciassociales,NB,{'priors': [None]},0.7199,"[CalPromedioPrepa, Puntaje Seccion 21 Examen, ..."
0,ambienteconstruido,NB,{'priors': [None]},0.7542,"[CalPromedioPrepa, Eva_ini_UMF1, Eva_ini_UMM2,..."


In [21]:
repeat_list[1].head(6)

,Area,Model,Parameters,Accuracy,Variables
0,ingenieria,LogReg,NA,0.7256,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,negocios,LogReg,NA,0.7034,"[CalPromedioPrepa, Puntaje Seccion 8 Examen, P..."
0,estudioscreativos,SVM,{'kernel': ['linear']},0.6295,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,salud,XGB,"{'learning_rate': [0.05], 'max_depth': [8], 'm...",0.6648,"[CalPromedioPrepa, enojo_n]"
0,cienciassociales,NB,{'priors': [None]},0.7199,"[CalPromedioPrepa, Puntaje Seccion 21 Examen, ..."
0,ambienteconstruido,NB,{'priors': [None]},0.7542,"[CalPromedioPrepa, Eva_ini_UMF1, Eva_ini_UMM2,..."


In [20]:
repeat_list[2].head(6)

,Area,Model,Parameters,Accuracy,Variables
0,ingenieria,LogReg,NA,0.7256,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,negocios,LogReg,NA,0.7034,"[CalPromedioPrepa, Puntaje Seccion 8 Examen, P..."
0,estudioscreativos,SVM,{'kernel': ['linear']},0.6295,"[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,..."
0,salud,XGB,"{'learning_rate': [0.05], 'max_depth': [8], 'm...",0.6648,"[CalPromedioPrepa, enojo_n]"
0,cienciassociales,NB,{'priors': [None]},0.7199,"[CalPromedioPrepa, Puntaje Seccion 21 Examen, ..."
0,ambienteconstruido,NB,{'priors': [None]},0.7542,"[CalPromedioPrepa, Eva_ini_UMF1, Eva_ini_UMM2,..."


In [22]:
repeat_list[0].to_csv("C:/Users/Milara/...", encoding="ISO-8859-1", index = False) #thus it is necessary to save it as the usual type of coding

## **10: Comparing the EVAL NO CAL PREPA VS THE BEST EXPERIMENTS with an statistical test**

In [18]:
import pandas as pd

#Read the file with the data of the best experiments
df_dif = pd.read_csv("C:/Users/Milara/...",encoding="ISO-8859-1")
df_dif.head(10)

,Area,Model,Parameters,Accuracy,Variables
0,ingenieria,LogReg,NOT,0.7256,"['CalPromedioPrepa', 'Eva_ini_UMM1', 'Eva_ini_..."
1,negocios,LogReg,NOT,0.7034,"['CalPromedioPrepa', 'Puntaje Seccion 8 Examen..."
2,estudioscreativos,SVM,{'kernel': 'linear'},0.6295,"['CalPromedioPrepa', 'Eva_ini_UMM1', 'Eva_ini_..."
3,salud,XGB,"{'learning_rate': 0.05, 'max_depth': 8, 'min_c...",0.6648,"['CalPromedioPrepa', 'enojo_n']"
4,cienciassociales,NB,{'priors': None},0.7199,"['CalPromedioPrepa', 'Puntaje Seccion 21 Exame..."
5,ambienteconstruido,NB,{'priors': None},0.7542,"['CalPromedioPrepa', 'Eva_ini_UMF1', 'Eva_ini_..."
6,ingenieria,SVM,{'kernel': 'rbf'},0.6800,"['Eva_ini_UMM1', 'Eva_ini_UMM2', 'Eva_ini_UMF1..."
7,negocios,SVM,{'kernel': 'linear'},0.6100,"['Eva_ini_UMM1', 'Eva_ini_UMM2', 'Eva_ini_UMM3..."
8,estudioscreativos,XGB,"{'learning_rate': 0.01, 'max_depth': 8, 'min_c...",0.6200,"['Eva_ini_UMM2', 'Eva_ini_UMF1', 'Eva_ini_UMC1']"
9,salud,LogReg,NOT,0.6100,['Eva_ini_UMQ2']


### Functions to compare both types of experiments (based on previous code)

In [20]:
from scipy import stats
import random

def run_cv_2(df_inuse,id_area,noise_flag = False, coef_flag = False):
    """
    Function to run the experiments based on their parameters and noise and return the cv results
    Input: 
        df with the experiments to be run,
        value of id of the area
        boolean value for the flag
    Output:
        df with the cross validation scores
    """
    df_db = dflist_areas[id_area]# Get the full database
    

    #Store the data to configure the experiment
    models = df_inuse['Model'].tolist()
    variables = df_inuse['Variables'].tolist()
    parameters = df_inuse['Parameters'].tolist()
    
    list_results = list()

    #Compute the models
    for i in range(len(models)):
        list_variables = string_2list(variables[i]) # get the list of variables to be used in the model
        
        #get the parameters for the model
        if parameters[i] != 'NOT':
            dict_param = ast.literal_eval(parameters[i])

            for elem in dict_param.keys(): #transform to list the values of each key
                dict_param[elem] = [dict_param[elem]]
        else:
            dict_param = 'NA'

        #Clean the db with the selected variables
        data_samples = df_db.loc[:,list_variables]
        
        if noise_flag:#used only if flag enabled
            #np.random.seed(100)
            data_samples.insert(0,column = 'Noise', value = np.random.randint(1,1000,df_db.shape[0])) # Noise added
            
        X = preprocessing.scale(data_samples)
        y = df_db['Binary_out']
        model = models[i].lower() 

        #Compute the model
        if model == 'rf':
            rf_res = rf_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(rf_res)
        elif model == 'logreg':
            log_res = logreg_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist(), coefficients = coef_flag)
            list_results.append(log_res)
        elif model == 'svm':
            svm_res = svm_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist(), coefficients = coef_flag)
            list_results.append(svm_res)
        elif model == 'knn':
            knn_res = knn_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(knn_res)
        elif model == 'nb':
            nb_res = nb_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist())
            list_results.append(nb_res)
        elif model == 'xgb':
            xgb_res = xgb_model_S(X,y, areas[id_area], dict_param, data_samples.columns.tolist(), coefficients = coef_flag)
            list_results.append(xgb_res)

    return pd.concat(list_results, axis = 0)


def paired_ttest_2(base_cv,noise_cv):
    """
    Function that is used to compare the mean using paired t test
    Input: two df 
    Output: model with less difference in the means
    """
    statistic, pvalue = stats.ttest_rel(base_cv['CV Results'][0], noise_cv['CV Results'][0])
    return(pvalue)

### *Testing it*

In [46]:
areas = ['ingenieria', 'negocios','estudioscreativos', 'salud', 'cienciassociales', 'ambienteconstruido']
id_area = 0
winner_list = []

for i in range(6): #
    #Get the data for the experiment
    df_inuse = df_dif[df_dif['Area'] == areas[i]].reset_index().drop('index', axis = 1) # Select the area

    base_cv = run_cv_2(df_inuse.loc[[0]],i)
    eval_cv = run_cv_2(df_inuse.loc[[1]],i)
    
    #used to get the precision and recall
    print(areas[i],"AD metric: " + str(base_cv['Accuracy'].values))
    print(areas[i],"EvalOnly metric: " +str(eval_cv['Accuracy'].values))

    #print(areas[i], paired_ttest_2(base_cv,eval_cv)) # uncomment when comparing the models from same area

ingenieria AD metric: [0.7301]
ingenieria EvalOnly metric: [0.7131]
negocios AD metric: [0.7185]
negocios EvalOnly metric: [0.6297]
[13:24:29] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:24:29] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:24:30] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:24:30] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with t

# **11. Get coefficients**

In [69]:
areas = ['ingenieria', 'negocios','estudioscreativos', 'salud', 'cienciassociales', 'ambienteconstruido'] # 
id_area = 0
coeff_list = []

areas_withsd = [1,1,0,1,1,1] #Based on the results above of statistical difference

#Returns the experiment with admission data if there is significative diference between it and the evalcalnoprepa, if not it returns the one with less variables
for i in range(6): #
    #Get the data for the experiment
    df_inuse = df_dif[df_dif['Area'] == areas[i]].reset_index().drop('index', axis = 1) # Select the area

    base_cv = run_cv_2(df_inuse.loc[[0]],i,coef_flag = True)
    eval_cv = run_cv_2(df_inuse.loc[[1]],i,coef_flag = True)
    
    if areas_withsd[i]:
        coeff_list.append(base_cv)
    else:
        if len(base_cv['Variables'].values) < len(eval_cv['Variables'].values):
            coeff_list.append(base_cv)
        else:
            coeff_list.append(eval_cv)

    print(areas[i])
    
coefficients_df = pd.concat(coeff_list,axis =0)
coefficients_df.head(6)

ingenieria
negocios
estudioscreativos
salud
cienciassociales
ambienteconstruido


,Area,Model,Parameters,Accuracy,CV Results,Variables,Coefficients
0,ingenieria,LogReg,NA,0.7256,"[0.7165775401069518, 0.7058823529411765, 0.736...","[CalPromedioPrepa, Eva_ini_UMM1, Eva_ini_UMM2,...","[[0.6482572642354945, 0.23564007679801977, 0.2..."
0,negocios,LogReg,NA,0.7034,"[0.6422018348623854, 0.7064220183486238, 0.715...","[CalPromedioPrepa, Puntaje Seccion 8 Examen, P...","[[0.6936451310090773, 0.27687448870875303, 0.1..."
0,estudioscreativos,XGB,"{'learning_rate': [0.01], 'max_depth': [8], 'm...",0.6182,"[0.6818181818181818, 0.5, 0.5454545454545454, ...","[Eva_ini_UMM2, Eva_ini_UMF1, Eva_ini_UMC1]",[]
0,salud,XGB,"{'learning_rate': [0.05], 'max_depth': [8], 'm...",0.6648,"[0.6666666666666666, 0.6666666666666666, 0.583...","[CalPromedioPrepa, enojo_n]",[]
0,cienciassociales,NB,{'priors': [None]},0.7199,"[0.7619047619047619, 0.6666666666666666, 0.619...","[CalPromedioPrepa, Puntaje Seccion 21 Examen, ...",NOT
0,ambienteconstruido,NB,{'priors': [None]},0.7542,"[0.5833333333333334, 0.8333333333333334, 0.75,...","[CalPromedioPrepa, Eva_ini_UMF1, Eva_ini_UMM2,...",NOT


In [22]:
coefficients_df.to_csv("C:/Users/Milara/...", encoding="ISO-8859-1", index = False) #thus it is necessary to save it as the usual type of coding

### Manual extraction of feature importance

In [40]:
#Manual feature importances of XGB
import pandas as pd

#Read the file with the data of the best experiments
df_coef = pd.read_csv("C:/Users/Milara/...",encoding="ISO-8859-1")

areaCoeff = 'salud'#['estudioscreativos','salud']
areaCoeff_id = 3 #[2,3]


df_inuse = df_coef[df_coef['Area'] == areaCoeff]
df_db = dflist_areas[areaCoeff_id]# Get the full database

variables = df_inuse['Variables'].tolist()
list_variables = string_2list(variables[0]) # get the list of variables to be used in the model
#Clean the db with the selected variables
data_samples = df_db.loc[:,list_variables]

X = preprocessing.scale(data_samples)
y = df_db['Binary_out']

#params of estudios creativos {'learning_rate': [0.01], 'max_depth': [8], 'min_child_weight': [6], 'n_estimators': [100], 'random_state': [17], 'subsample': [0.5]}
#params of salud              {'learning_rate': [0.05], 'max_depth': [8], 'min_child_weight': [6], 'n_estimators': [100], 'random_state': [17], 'subsample': [0.5]}

xgb_print = xgb.XGBClassifier(learning_rate= 0.05, max_depth=  8, min_child_weight=  6, use_label_encoder = False, 
                              n_estimators=  100, random_state=  17, subsample= 0.5).fit(X, y)
coefficients_xgb = xgb_print.feature_importances_

print(areaCoeff)
print(variables)
print(coefficients_xgb)


[13:21:37] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
salud
["['CalPromedioPrepa', 'enojo_n']"]
[0.34992912 0.6500709 ]
